In [1]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from xgboost import plot_tree, plot_importance


from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:

data = pd.read_csv('/Users/pierros/Documents/Courses/ECE6254/Projects/Final Project/features15sec.csv')
data = data.iloc[0:, 1:] 
data.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,pop.00027.wav,15.0,0.379576,0.088084,0.150022,0.002675,2587.140475,8.744579e+05,2956.302829,198766.050606,...,1.0,8.857656e-09,1.0,1.476276e-09,1.0,-1.181021e-08,1.0,8.857656e-09,1.0,pop
1,pop.00033.wav,15.0,0.393467,0.093529,0.124740,0.003191,2756.163230,6.958309e+05,2616.865245,244235.670584,...,1.0,5.905104e-09,1.0,5.905104e-09,1.0,-1.181021e-08,1.0,0.000000e+00,1.0,pop
2,pop.00032.wav,15.0,0.421139,0.092977,0.095513,0.002106,4330.352239,1.974231e+06,3457.327124,112476.744036,...,1.0,0.000000e+00,1.0,0.000000e+00,1.0,5.905104e-09,1.0,5.905104e-09,1.0,pop
3,pop.00026.wav,15.0,0.315736,0.086414,0.198041,0.007539,2483.616949,8.635061e+05,2836.591500,251765.001210,...,1.0,0.000000e+00,1.0,-5.905104e-09,1.0,-1.181021e-08,1.0,1.181021e-08,1.0,pop
4,pop.00030.wav,15.0,0.375461,0.081815,0.200531,0.006887,2982.381296,5.960961e+05,2924.271786,133121.273287,...,1.0,8.857656e-09,1.0,1.476276e-08,1.0,2.952552e-09,1.0,0.000000e+00,1.0,pop


In [3]:
data = data.drop(["filename","length"], axis=1)

In [4]:
y = data['label'] # genre variable.
X = data.loc[:, data.columns != 'label'] #select all columns but not the labels

#### NORMALIZE X ####

# Normalize so everything is on the same scale. 

cols = X.columns
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)

# new data frame with the new scaled data. 
X = pd.DataFrame(np_scaled, columns = cols)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Standardize Data

In [6]:
scaler = preprocessing.StandardScaler(with_mean=True, with_std=True).fit(X_train)

X_train_stand = scaler.transform(X_train)
X_test_stand = scaler.transform(X_test)

In [7]:
def model_assess(model, title = "Default"):
    model.fit(X_train_stand, y_train)
    preds = model.predict(X_test_stand)
    #print(confusion_matrix(y_test, preds))
    print('Accuracy', title, ':', round(accuracy_score(y_test, preds), 5), '\n')
    return round(accuracy_score(y_test, preds), 5)

Native Bayes

In [8]:
nb = GaussianNB()
score = model_assess(nb, "Naive Bayes")

Accuracy Naive Bayes : 0.44 



Stochastic Gradient Descent

In [9]:
sgd = SGDClassifier(max_iter=500, random_state=0)
score = model_assess(sgd, "Stochastic Gradient Descent")

Accuracy Stochastic Gradient Descent : 0.41667 



KNN

In [10]:
knn_scores = []

for k in range(1,500):
    knn = KNeighborsClassifier(n_neighbors=k)
    # print(k)
    score = model_assess(knn, "KNN")
    knn_scores.append(score)

# Find index with the lowest error
df_knn = pd.DataFrame(knn_scores)
df_knn.index = np.arange(1, len(df_knn) + 1)
best_k = df_knn.idxmax()[0]
print('Best K:',best_k, 'Value:',df_knn.max()[0])


Accuracy KNN : 0.30667 

Accuracy KNN : 0.33667 

Accuracy KNN : 0.32667 

Accuracy KNN : 0.32667 

Accuracy KNN : 0.36333 

Accuracy KNN : 0.37333 

Accuracy KNN : 0.36667 

Accuracy KNN : 0.35333 

Accuracy KNN : 0.34333 

Accuracy KNN : 0.33333 

Accuracy KNN : 0.36 

Accuracy KNN : 0.36667 

Accuracy KNN : 0.35 

Accuracy KNN : 0.33333 

Accuracy KNN : 0.34 

Accuracy KNN : 0.37 

Accuracy KNN : 0.35333 

Accuracy KNN : 0.36 

Accuracy KNN : 0.35667 

Accuracy KNN : 0.35667 

Accuracy KNN : 0.34 

Accuracy KNN : 0.35 

Accuracy KNN : 0.36333 

Accuracy KNN : 0.33333 

Accuracy KNN : 0.34 

Accuracy KNN : 0.33333 

Accuracy KNN : 0.33333 

Accuracy KNN : 0.34333 

Accuracy KNN : 0.35 

Accuracy KNN : 0.35333 

Accuracy KNN : 0.36333 

Accuracy KNN : 0.34667 

Accuracy KNN : 0.34333 

Accuracy KNN : 0.34667 

Accuracy KNN : 0.34667 

Accuracy KNN : 0.35 

Accuracy KNN : 0.34333 

Accuracy KNN : 0.35 

Accuracy KNN : 0.35667 

Accuracy KNN : 0.35333 

Accuracy KNN : 0.35667 

Accuracy

In [11]:
best_k

73

In [12]:
# With tuned parameter
knn = KNeighborsClassifier(n_neighbors=best_k)
score = model_assess(knn, "KNN")

Accuracy KNN : 0.37667 



Decission trees

In [13]:
tree = DecisionTreeClassifier()
score = model_assess(tree, "Decission trees")

Accuracy Decission trees : 0.42333 



Random Forest

In [15]:
rf_scores = []
for est in tqdm(range(1000,3000,100)):
    d_scores = []
    for d in range(1,1000,100):
        rforest = RandomForestClassifier(n_estimators=est, max_depth=d, random_state=0)
        score = model_assess(rforest, "Random Forest")
        d_scores.append(score)
    rf_scores.append(d_scores)

df_rf = pd.DataFrame(rf_scores)

best_rf_param = np.argwhere(rf_scores == np.max(rf_scores))[0]

print("Best n_est:",best_rf_param[0],"Best depth:",best_rf_param[1])

  0%|          | 0/20 [00:00<?, ?it/s]

Accuracy Random Forest : 0.31 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 



  5%|▌         | 1/20 [00:19<06:10, 19.48s/it]

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.31333 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 

Accuracy Random Forest : 0.6 



 10%|█         | 2/20 [00:48<07:29, 24.95s/it]

...

Accuracy Random Forest : 0.59 

Accuracy Random Forest : 0.31667 

Accuracy Random Forest : 0.59 

Accuracy Random Forest : 0.59 

Accuracy Random Forest : 0.59 

Accuracy Random Forest : 0.59 

Accuracy Random Forest : 0.59 

Accuracy Random Forest : 0.59 

Accuracy Random Forest : 0.59 

Accuracy Random Forest : 0.59 



 95%|█████████▌| 19/20 [14:28<01:01, 61.56s/it]

Accuracy Random Forest : 0.59 

Accuracy Random Forest : 0.31667 

Accuracy Random Forest : 0.59 

Accuracy Random Forest : 0.59 

Accuracy Random Forest : 0.59 

Accuracy Random Forest : 0.59 

Accuracy Random Forest : 0.59 

Accuracy Random Forest : 0.59 

Accuracy Random Forest : 0.59 

Accuracy Random Forest : 0.59 



100%|██████████| 20/20 [15:37<00:00, 46.89s/it]

Accuracy Random Forest : 0.59 

Best n_est: 0 Best depth: 1


In [16]:
print(best_rf_param[0]+1, best_rf_param[1]+1)


1 2


In [17]:
# With tuned parameters
rforest = RandomForestClassifier(n_estimators=best_rf_param[0]+1, max_depth=best_rf_param[1]+1, random_state=0)
score = model_assess(rforest, "Random Forest")

Accuracy Random Forest : 0.18 



Support Vector Machine

In [18]:
svm = SVC(decision_function_shape="ovo")
score = model_assess(svm, "Support Vector Machine")
svm = SVC(decision_function_shape="ovr")
score = model_assess(svm, "Support Vector Machine")


Accuracy Support Vector Machine : 0.47667 

Accuracy Support Vector Machine : 0.47667 



Logistic Regression

In [19]:
lg = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
score = model_assess(lg, "Logistic Regression")

Accuracy Logistic Regression : 0.46333 



/Users/pierros/opt/anaconda3/envs/keras_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Neural Nets

In [20]:
best_score_mlp = 0
for a in np.linspace(0.00001,0.0001,20):
    nn = MLPClassifier(solver='lbfgs', alpha=a, hidden_layer_sizes=(5000, 10), random_state=1)
    score = model_assess(nn, "Neural Nets")
    if score > best_score_mlp:
        best_score_mlp = score
        best_alpha = a

Accuracy Neural Nets : 0.39333 

Accuracy Neural Nets : 0.39333 

Accuracy Neural Nets : 0.39 

Accuracy Neural Nets : 0.39333 

Accuracy Neural Nets : 0.39 

Accuracy Neural Nets : 0.39 

Accuracy Neural Nets : 0.39 

Accuracy Neural Nets : 0.39333 

Accuracy Neural Nets : 0.39667 

Accuracy Neural Nets : 0.39 

Accuracy Neural Nets : 0.39333 

Accuracy Neural Nets : 0.39667 

Accuracy Neural Nets : 0.39 

Accuracy Neural Nets : 0.39667 

Accuracy Neural Nets : 0.39 

Accuracy Neural Nets : 0.39 

Accuracy Neural Nets : 0.39333 

Accuracy Neural Nets : 0.38667 

Accuracy Neural Nets : 0.39 

Accuracy Neural Nets : 0.38667 



In [21]:
print(best_alpha)

4.789473684210527e-05


In [22]:
nn = MLPClassifier(solver='lbfgs', alpha=best_alpha, hidden_layer_sizes=(5000, 10), random_state=1)
score = model_assess(nn, "Neural Nets")

Accuracy Neural Nets : 0.39667 



In [23]:
# Transform Data for the next classifiers
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train2 = le.fit_transform(y_train)
y_test2 =le.fit_transform(y_test)

Cross Gradient Booster

In [24]:
best_cgb = [1,0.01]
best_score_cgb = 0
for est in tqdm(range(1000,2000,100)):
    for lr in np.linspace(0.1,0.7,7):
        xgb = XGBClassifier(n_estimators=est, learning_rate=lr)
        # model_assess(xgb, "Cross Gradient Booster")
        xgb.fit(X_train_stand, y_train2)
        preds = xgb.predict(X_test_stand)
        #print(confusion_matrix(y_test, preds))
        print('Accuracy', "Cross Gradient Booster", ':', round(accuracy_score(y_test2, preds), 5), '\n')
        if round(accuracy_score(y_test2, preds), 5) > best_score_cgb:
            best_score_cgb = round(accuracy_score(y_test2, preds), 5)
            best_cgb = [est, lr]

  0%|          | 0/10 [00:00<?, ?it/s]

Accuracy Cross Gradient Booster : 0.57667 

Accuracy Cross Gradient Booster : 0.56333 

Accuracy Cross Gradient Booster : 0.56333 

Accuracy Cross Gradient Booster : 0.57 

Accuracy Cross Gradient Booster : 0.57 

Accuracy Cross Gradient Booster : 0.56667 



 10%|█         | 1/10 [52:59<7:56:55, 3179.55s/it]

Accuracy Cross Gradient Booster : 0.56333 

Accuracy Cross Gradient Booster : 0.57667 

Accuracy Cross Gradient Booster : 0.56 

Accuracy Cross Gradient Booster : 0.56667 

Accuracy Cross Gradient Booster : 0.57 

Accuracy Cross Gradient Booster : 0.57 

Accuracy Cross Gradient Booster : 0.57 



 20%|██        | 2/10 [1:01:49<3:36:06, 1620.86s/it]

...

Accuracy Cross Gradient Booster : 0.56333 

Accuracy Cross Gradient Booster : 0.58333 

Accuracy Cross Gradient Booster : 0.56 

Accuracy Cross Gradient Booster : 0.56667 

Accuracy Cross Gradient Booster : 0.57333 

Accuracy Cross Gradient Booster : 0.57667 

Accuracy Cross Gradient Booster : 0.56667 



 90%|█████████ | 9/10 [2:18:56<12:18, 738.02s/it]

Accuracy Cross Gradient Booster : 0.56333 

Accuracy Cross Gradient Booster : 0.58333 

Accuracy Cross Gradient Booster : 0.55667 

Accuracy Cross Gradient Booster : 0.56667 

Accuracy Cross Gradient Booster : 0.57667 

Accuracy Cross Gradient Booster : 0.57667 

Accuracy Cross Gradient Booster : 0.57 



100%|██████████| 10/10 [2:31:20<00:00, 908.03s/it]

Accuracy Cross Gradient Booster : 0.56333 



In [25]:
best_cgb

[1700, 0.1]

In [26]:
xgb = XGBClassifier(n_estimators=best_cgb[0], learning_rate=best_cgb[1])
xgb.fit(X_train_stand, y_train2)
preds = xgb.predict(X_test_stand)
print('Accuracy', "Cross Gradient Booster", ':', round(accuracy_score(y_test2, preds), 5), '\n')

Accuracy Cross Gradient Booster : 0.58333 



Cross Gradient Booster (Random Forest)

In [27]:
xgbrf = XGBRFClassifier(objective= 'multi:softmax')
# model_assess(xgbrf, "Cross Gradient Booster (Random Forest)")
xgbrf.fit(X_train_stand, y_train2)
preds = xgbrf.predict(X_test_stand)
print('Accuracy', "Cross Gradient Booster (Random Forest)", ':', round(accuracy_score(y_test2, preds), 5), '\n')

Accuracy Cross Gradient Booster (Random Forest) : 0.55667 



In [28]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(estimator=xgb, random_state=1)
perm.fit(X_test_stand, y_test2)

eli5.show_weights(estimator=perm, feature_names = X_test_stand.columns.tolist())

AttributeError: 'numpy.ndarray' object has no attribute 'columns'